# Sensitivity Analysis

After finding the optimal decision strategy, it's crucial to understand how sensitive our results are to changes in the input parameters. **Sensitivity analysis** helps identify which uncertainties have the greatest impact on the expected utility and, consequently, on the optimal decisions.

This notebook is going to be focused on generating plots for sensitivity analysis of the Oil Decision Problem.

In [14]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import numpy as np
import pyagrum as grum
import pyagrum.lib.notebook as gnb
from pyagrum import InfluenceDiagram

## 1 - Baseline

### 1.1 - Influence diagram structure

In [15]:
influence_diagram = InfluenceDiagram()

Q = influence_diagram.addChanceNode(grum.LabelizedVariable("Q", "Q", 0).addLabel('high').addLabel('medium').addLabel('low'))
R = influence_diagram.addChanceNode(grum.LabelizedVariable("R", "R", 0).addLabel('pass').addLabel('fail').addLabel('no_results'))
T = influence_diagram.addDecisionNode(grum.LabelizedVariable("T", "T", 0).addLabel('do').addLabel('not_do'))
B = influence_diagram.addDecisionNode(grum.LabelizedVariable("B", "B", 0).addLabel('buy').addLabel('not_buy'))
U = influence_diagram.addUtilityNode(grum.LabelizedVariable("U", "U", 0).addLabel('utility'))

influence_diagram.addArc("T", "R")
influence_diagram.addArc("T", "B") # memory arc
influence_diagram.addArc("T", "U")
influence_diagram.addArc("R", "B")
influence_diagram.addArc("B", "U")
influence_diagram.addArc("Q", "R")
influence_diagram.addArc("Q", "U")

gnb.sideBySide(influence_diagram, captions=["Oil field influence diagram"])

### 1.2 - Influence diagram parameters

In [16]:
q_cpt = pd.DataFrame({
    "Q": ["high", "medium", "low"],
    "Probability": [0.35, 0.45, 0.2]
})
# Set Q values as index for easier access
q_cpt = q_cpt.set_index('Q')

r_cpt = pd.DataFrame({
    "R | Q": ["pass", "fail"],
    "high": [0.95, 0.05],
    "medium": [0.7, 0.3],
    "low": [0.15, 0.85]
})
# Set R | Q values as index for easier access
r_cpt = r_cpt.set_index('R | Q')

u_table = pd.DataFrame(
    [
        ['do', 'buy', 'high', 1220],
        ['do', 'buy', 'medium', 600],
        ['do', 'buy', 'low', -30],
        ['do', 'not_buy', '-', 320],
        ['not_do', 'buy', 'high', 1250],
        ['not_do', 'buy', 'medium', 630],
        ['not_do', 'buy', 'low', 0],
        ['not_do', 'not_buy', '-', 350],
    ],
    columns=['T', 'B', 'Q', 'U']
)

# Q probabilities
influence_diagram.cpt(Q)[:] = q_cpt['Probability'].to_list()

# R probabilities
for q in ["high", "medium", "low"]:
    pass_prob = r_cpt.loc['pass', q]
    fail_prob = r_cpt.loc['fail', q]

    influence_diagram.cpt(R)[{"Q": q, "T": "do"}] = [pass_prob, fail_prob, 0.0]
    influence_diagram.cpt(R)[{"Q": q, "T": "not_do"}] = [0.0, 0.0, 1.0]

# U utility values
influence_diagram.utility(U)[{"T": "do", "B": "buy"}] = np.array(
    [u_table.iloc[0, 3], u_table.iloc[1, 3], u_table.iloc[2, 3]])[:, np.newaxis]
influence_diagram.utility(U)[{"T": "do", "B": "not_buy"}] = np.array(
    [u_table.iloc[3, 3]] * 3)[:, np.newaxis]
influence_diagram.utility(U)[{"T": "not_do", "B": "buy"}] = np.array(
    [u_table.iloc[4, 3], u_table.iloc[5, 3], u_table.iloc[6, 3]])[:, np.newaxis]
influence_diagram.utility(U)[{"T": "not_do", "B": "not_buy"}] = np.array(
    [u_table.iloc[7, 3]] * 3)[:, np.newaxis]

In [17]:
q_cpt

,Probability
Q,
high,0.35
medium,0.45
low,0.20


In [18]:
r_cpt

,high,medium,low
R | Q,,,
pass,0.95,0.7,0.15
fail,0.05,0.3,0.85


### 1.3 - Baseline Optimal decision

In [19]:
inference_engine = grum.ShaferShenoyLIMIDInference(influence_diagram)
inference_engine.makeInference()
meu_result = inference_engine.MEU()
baseline_meu = meu_result['mean']

print(inference_engine.optimalDecision("B").argmax())
print(inference_engine.optimalDecision("T").argmax())

print(baseline_meu)

([{'B': 0, 'R': 0}, {'B': 0, 'R': 1}, {'B': 0, 'R': 2}], 1.0)
([{'T': 1}], 1.0)
721.0


## 2 - Single Parameter Sensitivity Analysis

### 2.1 - Framework

In [20]:
def validate_probability_changes(
        cpt_df: pd.DataFrame, 
        row_index: str, 
        column: str, 
        changes: list
):
    """
    Validates that probability increments are mathematically valid.
    
    For Q CPT: Ensures the entire probability distribution sums to 1
    For R CPT: Ensures each conditional distribution (column) sums to 1
    
    Parameters:
    - cpt_df: DataFrame with probabilities (index should be row identifiers)
    - row_index: Index value identifying the row to modify
    - column: Column name to modify
    - changes: List of incremental changes to apply (can be positive or negative)
    
    Returns:
    - List of valid changes
    
    Raises:
    - ValueError if any change would create invalid probabilities
    """
    if row_index not in cpt_df.index:
        raise ValueError(f"Row index '{row_index}' not found in CPT")
    
    if column not in cpt_df.columns:
        raise ValueError(f"Column '{column}' not found in CPT")
    
    # Get current probability value to modify
    current_value = cpt_df.loc[row_index, column]
    
    valid_changes = []
    invalid_changes = []
    
    for change in changes:
        # Calculate new probability value (always incremental)
        new_value = current_value + change
        change_desc = f"increment by {change}" if change >= 0 else f"decrement by {abs(change)}"
        
        # Check probability bounds: must be between 0 and 1
        if not (0 <= new_value <= 1):
            invalid_changes.append(f"Change '{change_desc}' results in invalid probability {new_value:.3f} (must be between 0 and 1)")
            continue
        
        # Check if remaining probability for redistribution is valid
        remaining_prob = 1 - new_value
        if remaining_prob < 0:
            invalid_changes.append(f"Change '{change_desc}' would require negative remaining probability {remaining_prob:.3f}")
            continue
        
        valid_changes.append(change)
    
    # Raise detailed error if any changes are invalid
    if invalid_changes:
        error_msg = f"Invalid probability changes detected for {row_index}[{column}] (current value: {current_value:.3f}):\n"
        for i, error in enumerate(invalid_changes, 1):
            error_msg += f"  {i}. {error}\n"
        error_msg += f"\nValid probability changes must result in values between 0 and 1, and allow for valid redistribution of remaining probability."
        raise ValueError(error_msg)
    
    if not valid_changes:
        raise ValueError(f"No valid changes provided for {row_index}[{column}]")
    
    return valid_changes

def generate_modified_cpts(cpt_df: pd.DataFrame, row_index: str, column: str, 
                          changes: list, redistribution: str = 'proportional'):
    """
    Creates multiple modified versions of a CPT with different probability values.
    
    Parameters:
    - cpt_df: Original CPT DataFrame (with appropriate index)
    - row_index: Index value identifying the row to modify
    - column: Column name to modify
    - changes: List of incremental changes to apply
    - redistribution: 'proportional' or 'equal' for redistributing remaining probability
    
    Returns:
    - List of modified CPT DataFrames
    - List of actual parameter values used
    """
    modified_cpts = []
    parameter_values = []
    
    # Validate changes first
    valid_changes = validate_probability_changes(cpt_df, row_index, column, changes)
    
    for change in valid_changes:
        modified_cpt = cpt_df.copy()
        
        # Apply the probability change to the target row
        current_value = cpt_df.loc[row_index, column]
        new_value = current_value + change
        
        modified_cpt.loc[row_index, column] = new_value
        parameter_values.append(new_value)
        
        # Redistribute remaining probability among other rows to maintain column sum = 1
        other_rows = [idx for idx in cpt_df.index if idx != row_index]
        remaining_prob = 1 - new_value
        
        if len(other_rows) > 0:
            if redistribution == 'proportional':
                # Maintain relative proportions among other probability values
                current_other_sum = cpt_df.loc[other_rows, column].sum()
                if current_other_sum > 0:
                    scale_factor = remaining_prob / current_other_sum
                    for row in other_rows:
                        modified_cpt.loc[row, column] = cpt_df.loc[row, column] * scale_factor
                else:
                    # Fallback to equal distribution if sum is zero
                    for row in other_rows:
                        modified_cpt.loc[row, column] = remaining_prob / len(other_rows)
            else:  # equal redistribution
                # Distribute remaining probability equally among other rows
                for row in other_rows:
                    modified_cpt.loc[row, column] = remaining_prob / len(other_rows)
        
        modified_cpts.append(modified_cpt)
    
    return modified_cpts, parameter_values

def calculate_meu_for_modified_cpts(modified_q_cpts=None, modified_r_cpts=None, 
                                   base_q_cpt=None, base_r_cpt=None):
    """
    Calculate MEU for each modified CPT configuration.
    
    Parameters:
    - modified_q_cpts: List of modified Q CPTs (None if Q is not being varied)
    - modified_r_cpts: List of modified R CPTs (None if R is not being varied) 
    - base_q_cpt: Base Q CPT to use when Q is not being varied
    - base_r_cpt: Base R CPT to use when R is not being varied
    
    Returns:
    - List of MEU values
    - List of optimal decisions
    """
    meu_values = []
    optimal_decisions = []
    
    # Determine which CPTs are being modified
    if modified_q_cpts is not None:
        cpt_list = modified_q_cpts
        varying_q = True
        q_base = base_q_cpt if base_q_cpt is not None else q_cpt
        r_base = base_r_cpt if base_r_cpt is not None else r_cpt
    else:
        cpt_list = modified_r_cpts
        varying_q = False
        q_base = base_q_cpt if base_q_cpt is not None else q_cpt
        r_base = base_r_cpt if base_r_cpt is not None else r_cpt
    
    for i, modified_cpt in enumerate(cpt_list):
        # Create fresh influence diagram for each parameter value
        temp_diagram = InfluenceDiagram()
        
        # Rebuild the same network structure as the original
        Q_temp = temp_diagram.addChanceNode(grum.LabelizedVariable("Q", "Q", 0).addLabel('high').addLabel('medium').addLabel('low'))
        R_temp = temp_diagram.addChanceNode(grum.LabelizedVariable("R", "R", 0).addLabel('pass').addLabel('fail').addLabel('no_results'))
        T_temp = temp_diagram.addDecisionNode(grum.LabelizedVariable("T", "T", 0).addLabel('do').addLabel('not_do'))
        B_temp = temp_diagram.addDecisionNode(grum.LabelizedVariable("B", "B", 0).addLabel('buy').addLabel('not_buy'))
        U_temp = temp_diagram.addUtilityNode(grum.LabelizedVariable("U", "U", 0).addLabel('utility'))
        
        # Define the same dependencies and information flow
        temp_diagram.addArc("T", "R")
        temp_diagram.addArc("T", "B")
        temp_diagram.addArc("T", "U")
        temp_diagram.addArc("R", "B")
        temp_diagram.addArc("B", "U")
        temp_diagram.addArc("Q", "R")
        temp_diagram.addArc("Q", "U")
        
        # Set probabilities
        if varying_q:
            # Use modified Q CPT and base R CPT
            temp_diagram.cpt(Q_temp)[:] = modified_cpt['Probability'].to_list()
            current_r_cpt = r_base
        else:
            # Use base Q CPT and modified R CPT  
            temp_diagram.cpt(Q_temp)[:] = q_base['Probability'].to_list()
            current_r_cpt = modified_cpt
        
        # Set R probabilities
        for q in ["high", "medium", "low"]:
            pass_prob = current_r_cpt.loc['pass', q]
            fail_prob = current_r_cpt.loc['fail', q]
            
            temp_diagram.cpt(R_temp)[{"Q": q, "T": "do"}] = [pass_prob, fail_prob, 0.0]
            temp_diagram.cpt(R_temp)[{"Q": q, "T": "not_do"}] = [0.0, 0.0, 1.0]
        
        # Set utility values (same as original)
        temp_diagram.utility(U_temp)[{"T": "do", "B": "buy"}] = np.array(
            [u_table.iloc[0, 3], u_table.iloc[1, 3], u_table.iloc[2, 3]])[:, np.newaxis]
        temp_diagram.utility(U_temp)[{"T": "do", "B": "not_buy"}] = np.array(
            [u_table.iloc[3, 3]] * 3)[:, np.newaxis]
        temp_diagram.utility(U_temp)[{"T": "not_do", "B": "buy"}] = np.array(
            [u_table.iloc[4, 3], u_table.iloc[5, 3], u_table.iloc[6, 3]])[:, np.newaxis]
        temp_diagram.utility(U_temp)[{"T": "not_do", "B": "not_buy"}] = np.array(
            [u_table.iloc[7, 3]] * 3)[:, np.newaxis]
        
        # Calculate MEU
        temp_inference = grum.ShaferShenoyLIMIDInference(temp_diagram)
        temp_inference.makeInference()
        meu_result = temp_inference.MEU()
        meu_values.append(meu_result['mean'])
        
        # Get optimal decisions
        optimal_T = temp_inference.optimalDecision("T").argmax()
        optimal_B_tensor = temp_inference.optimalDecision("B")
        
        # For T decision (simple decision)
        t_decision = 'do' if optimal_T[0][0]['T'] == 0 else 'not_do'
        
        # For B decision (depends on R), extract the policy
        b_instantiations, _ = optimal_B_tensor.argmax()
        b_policy = {}
        for inst in b_instantiations:
            r_value = inst['R']
            b_value = inst['B']
            r_label = ['pass', 'fail', 'no_results'][r_value]
            b_label = 'buy' if b_value == 0 else 'not_buy'
            b_policy[r_label] = b_label
        
        optimal_decisions.append({
            'T': t_decision,
            'B': b_policy
        })
    
    return meu_values, optimal_decisions


def plot_sensitivity_analysis(parameter_values: list, meu_values: list, 
                             parameter_name: str, baseline_meu: float = None,
                             optimal_decisions: list = None):
    """
    Plot sensitivity analysis results with MEU on X-axis and parameter probability on Y-axis using Plotly.
    
    Parameters:
    - parameter_values: Y-axis values (parameter probability values tested)
    - meu_values: X-axis values (corresponding MEU values)
    - parameter_name: Name of parameter being analyzed
    - baseline_meu: Original MEU value for reference line
    - optimal_decisions: List of optimal decision dictionaries for hover info
    """
    # Create hover text with optimal decisions
    hover_text = []
    if optimal_decisions:
        for i, decision in enumerate(optimal_decisions):
            b_policy_str = ', '.join([f"R={r}→{b}" for r, b in decision['B'].items()])
            hover_info = f"Parameter: {parameter_values[i]:.3f}<br>MEU: {meu_values[i]:.1f}<br>T: {decision['T']}<br>B: [{b_policy_str}]"
            hover_text.append(hover_info)
    else:
        hover_text = [f"Parameter: {param:.3f}<br>MEU: {meu:.1f}" 
                     for param, meu in zip(parameter_values, meu_values)]
    
    # Create the main scatter plot
    fig = go.Figure()
    
    # Add scatter points with line
    fig.add_trace(go.Scatter(
        x=meu_values,
        y=parameter_values,
        mode='markers+lines',
        marker=dict(size=12, color='steelblue', line=dict(width=2, color='navy')),
        line=dict(width=3, color='steelblue'),
        hovertemplate='%{hovertext}<extra></extra>',
        hovertext=hover_text,
        name='Sensitivity Analysis'
    ))
    
    # Add MEU value annotations (only MEU, not parameter)
    for i, (x, y) in enumerate(zip(meu_values, parameter_values)):
        fig.add_annotation(
            x=x, y=y,
            text=f'{x:.1f}',
            showarrow=False,
            xshift=15, yshift=5,
            font=dict(size=10, color='black'),
            bgcolor='rgba(255,255,255,0.8)',
            bordercolor='black',
            borderwidth=1
        )
    
    # Add baseline reference line if provided
    if baseline_meu is not None:
        fig.add_vline(
            x=baseline_meu, 
            line_dash="dash", 
            line_color="red",
            annotation_text=f"Baseline MEU: {baseline_meu:.1f}",
            annotation_position="top"
        )
    
    # Update layout
    fig.update_layout(
        title=f'Sensitivity Analysis: {parameter_name} vs MEU',
        xaxis_title='Maximum Expected Utility (MEU)',
        yaxis_title=parameter_name,
        width=800,
        height=600,
        showlegend=False,
        template="plotly_white"
    )
    
    fig.show()

    return fig


def print_sensitivity_results(results: dict):
    """
    Print sensitivity analysis results in a formatted way.
    
    Parameters:
    - results: Dictionary returned from sensitivity_analysis function
    """
    print(f"\n📈 Sensitivity Analysis Results:")
    print(f"Parameter: {results['parameter_name']}")
    print(f"Baseline MEU: {results['baseline_meu']:.1f}")
    print(f"\nParameter Value → MEU:")
    for param_val, meu_val in zip(results['parameter_values'], results['meu_values']):
        print(f"  {param_val:.3f} → {meu_val:.1f}")

    print(f"\n🎯 Optimal Decisions:")
    for i, (param_val, decision) in enumerate(zip(results['parameter_values'], results['optimal_decisions'])):
        b_policy_str = ', '.join([f"R={r}→{b}" for r, b in decision['B'].items()])
        print(f"  P(param)={param_val:.3f}: T={decision['T']}, B=[{b_policy_str}]")


def plot_sensitivity_results(results: dict):
    """
    Plot sensitivity analysis results using the computed results.
    
    Parameters:
    - results: Dictionary returned from sensitivity_analysis function
    """
    fig =plot_sensitivity_analysis(
        results['parameter_values'], 
        results['meu_values'], 
        results['parameter_name'], 
        results['baseline_meu'],
        results['optimal_decisions']
    )

    return fig


def sensitivity_analysis(
    cpt_df: pd.DataFrame, 
    row_index: str, 
    column: str, 
    changes: list, 
    baseline_meu: float,
    redistribution: str = 'proportional'
):
    """
    Complete sensitivity analysis workflow for a single parameter.
    
    Parameters:
    - cpt_df: CPT DataFrame to modify (q_cpt or r_cpt)
    - row_index: Row identifier (e.g., 'high' for Q, 'pass' for R)
    - column: Column name to modify (e.g., 'Probability' for Q, 'high' for R)
    - changes: List of incremental changes to test (can be positive or negative)
    - baseline_meu: Baseline MEU value for reference
    - redistribution: 'proportional' or 'equal'
    
    Returns:
    - Dictionary with results including parameter values, MEU values, and optimal decisions
    """
    # Determine if we're working with Q or R CPT
    is_q_cpt = 'Probability' in cpt_df.columns
    
    # Generate modified CPTs
    modified_cpts, parameter_values = generate_modified_cpts(
        cpt_df, row_index, column, changes, redistribution
    )
    
    # Calculate MEU for each modified CPT
    if is_q_cpt:
        meu_values, optimal_decisions = calculate_meu_for_modified_cpts(
            modified_q_cpts=modified_cpts
        )
        param_name = f"P(Q={row_index})"
    else:
        meu_values, optimal_decisions = calculate_meu_for_modified_cpts(
            modified_r_cpts=modified_cpts
        )
        param_name = f"P(R={row_index} | Q={column})"
    
    # Return comprehensive results
    results = {
        'parameter_name': param_name,
        'parameter_values': parameter_values,
        'meu_values': meu_values,
        'optimal_decisions': optimal_decisions,
        'baseline_meu': baseline_meu,
        'modified_cpts': modified_cpts
    }
    
    return results

### 2.2 - Example 1: Q CPT Sensitivity Analysis

In [21]:
# Test sensitivity analysis on Q probability for 'high' oil quality
print("🔍 Testing sensitivity analysis on P(Q=high)")
print("=" * 50)

# Run sensitivity analysis (just computation)
q_results = sensitivity_analysis(
    cpt_df=q_cpt, 
    row_index='high', 
    column='Probability', 
    changes=[0, 0.05, 0.10, 0.15],
    baseline_meu=baseline_meu
)

# Display results using separate functions
print_sensitivity_results(q_results)

# Optionally plot results
fig = plot_sensitivity_results(q_results)

# Save the figure
html_content = pio.to_html(
        fig,
        include_plotlyjs='cdn',
        full_html=True,
        div_id="q-cpt-sensitivity-plot"
    )

print(html_content)

🔍 Testing sensitivity analysis on P(Q=high)

📈 Sensitivity Analysis Results:
Parameter: P(Q=high)
Baseline MEU: 721.0

Parameter Value → MEU:
  0.350 → 721.0
  0.400 → 761.7
  0.450 → 802.4
  0.500 → 843.1

🎯 Optimal Decisions:
  P(param)=0.350: T=not_do, B=[R=pass→buy, R=fail→buy, R=no_results→buy]
  P(param)=0.400: T=not_do, B=[R=pass→buy, R=fail→buy, R=no_results→buy]
  P(param)=0.450: T=not_do, B=[R=pass→buy, R=fail→buy, R=no_results→buy]
  P(param)=0.500: T=not_do, B=[R=pass→buy, R=fail→buy, R=no_results→buy]


<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-3.0.1.min.js" integrity="sha256-oy6Be7Eh6eiQFs5M7oXuPxxm9qbJXEtTpfSI93dW16Q=" crossorigin="anonymous"></script>                <div id="q-cpt-sensitivity-plot" class="plotly-graph-div" style="height:600px; width:800px;"></div>            <script type="text/javascript">                window.PLOTLYENV=window.PLOTLYENV || {};                                if (document.getElementById("q-cpt-sensitivity-plot")) {                    Plotly.newPlot(                        "q-cpt-sensitivity-plot",                        [{"hovertemplate":"%{hovertext}\u003cextra\u003e\u003c\u002fextra\u003e","hovertext":["Parameter: 0.350\u003cbr\u003eMEU: 721.0\u003cbr\u003eT: not_do\u003cbr\u003eB: [R=pass→buy, R=fail→buy, R=no_results→buy]","Parameter: 0.400\u003cbr\u003e

### 2.3 - Example 1: R CPT Sensitivity Analysis

In [22]:
print("\n" + "="*70)
print("🔍 Testing sensitivity analysis on P(R=pass | Q=high)")
print("="*70)

# Run sensitivity analysis on R CPT (test reliability for high quality fields)
r_results = sensitivity_analysis(
    cpt_df=r_cpt, 
    row_index='pass', 
    column='medium', 
    changes=[0, 0.05, 0.10, 0.15, 0.20, 0.25],
    baseline_meu=baseline_meu
)

# Display results
print_sensitivity_results(r_results)

# Plot results
fig = plot_sensitivity_results(r_results)

# Save the figure
html_content = pio.to_html(
        fig,
        include_plotlyjs='cdn',
        full_html=True,
        div_id="r-cpt-sensitivity-plot"
    )

print(html_content)


🔍 Testing sensitivity analysis on P(R=pass | Q=high)

📈 Sensitivity Analysis Results:
Parameter: P(R=pass | Q=medium)
Baseline MEU: 721.0

Parameter Value → MEU:
  0.700 → 721.0
  0.750 → 721.0
  0.800 → 721.0
  0.850 → 721.0
  0.900 → 722.1
  0.950 → 728.5

🎯 Optimal Decisions:
  P(param)=0.700: T=not_do, B=[R=pass→buy, R=fail→buy, R=no_results→buy]
  P(param)=0.750: T=not_do, B=[R=pass→buy, R=fail→buy, R=no_results→buy]
  P(param)=0.800: T=not_do, B=[R=pass→buy, R=fail→buy, R=no_results→buy]
  P(param)=0.850: T=not_do, B=[R=pass→buy, R=fail→buy, R=no_results→buy]
  P(param)=0.900: T=do, B=[R=pass→buy, R=fail→not_buy, R=no_results→buy]
  P(param)=0.950: T=do, B=[R=pass→buy, R=fail→not_buy, R=no_results→buy]


<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-3.0.1.min.js" integrity="sha256-oy6Be7Eh6eiQFs5M7oXuPxxm9qbJXEtTpfSI93dW16Q=" crossorigin="anonymous"></script>                <div id="r-cpt-sensitivity-plot" class="plotly-graph-div" style="height:600px; width:800px;"></div>            <script type="text/javascript">                window.PLOTLYENV=window.PLOTLYENV || {};                                if (document.getElementById("r-cpt-sensitivity-plot")) {                    Plotly.newPlot(                        "r-cpt-sensitivity-plot",                        [{"hovertemplate":"%{hovertext}\u003cextra\u003e\u003c\u002fextra\u003e","hovertext":["Parameter: 0.700\u003cbr\u003eMEU: 721.0\u003cbr\u003eT: not_do\u003cbr\u003eB: [R=pass→buy, R=fail→buy, R=no_results→buy]","Parameter: 0.750\u003cbr\u003e

In [ ]:
======================================================================
🔍 Testing sensitivity analysis on P(R=pass | Q=high)
======================================================================

📈 Sensitivity Analysis Results:
Parameter: P(R=pass | Q=medium)
Baseline MEU: 721.0

Parameter Value → MEU:
  0.700 → 721.0
  0.750 → 721.0
  0.800 → 721.0
  0.850 → 721.0
  0.900 → 722.1
  0.950 → 728.5

🎯 Optimal Decisions:
  P(param)=0.700: T=not_do, B=[R=pass→buy, R=fail→buy, R=no_results→buy]
  P(param)=0.750: T=not_do, B=[R=pass→buy, R=fail→buy, R=no_results→buy]
  P(param)=0.800: T=not_do, B=[R=pass→buy, R=fail→buy, R=no_results→buy]
  P(param)=0.850: T=not_do, B=[R=pass→buy, R=fail→buy, R=no_results→buy]
  P(param)=0.900: T=do, B=[R=pass→buy, R=fail→not_buy, R=no_results→buy]
  P(param)=0.950: T=do, B=[R=pass→buy, R=fail→not_buy, R=no_results→buy]

  asdasd